In [1]:
import csv
import os
import sys
import time
import pandas as pd
import numpy as np
from openpyxl import load_workbook

from selenium import webdriver

In [2]:
#currency = "USD" #美金
#currency = "CNY" #人民幣
#currency = "JPY" #日圓
#currency = "EUR" #歐元
#currency = "AUD" #澳幣
currency_list=["USD","CNY","JPY","EUR","AUD"]

In [3]:
for currency in currency_list:
    # Create a new instance of the Chrome driver
    driver = webdriver.Chrome()
    # Go to ESun Bank Historical Data page
    driver.get("https://www.esunbank.com.tw/bank/personal/deposit/rate/forex/exchange-rate-chart")

    # Wait the page to fresh.
    time.sleep(10)
    # Select the currency menu.
    currencyArrow = driver.find_element_by_css_selector(".transformSelect li span")
    currencyArrow.click()

    # Select the currency.
    currencyOptions = driver.find_elements_by_css_selector(".transformSelectDropdown li span")
    for c in currencyOptions:
        if currency in c.text:
            c.click()
            break

    # Wait the page to fresh.
    time.sleep(3)

    spotBtn = driver.find_element_by_css_selector(".radioBtns [for=\"spot\"]")
    spotBtn.click()

    # Wait the page to fresh.
    time.sleep(3)
    # Select the duration.
    durationBtn = driver.find_element_by_css_selector("div [for=\"oneYear\"]")
    durationBtn.click()

    # Wait the page to fresh.
    time.sleep(3)

    # Select the data type.
    dataBtn = driver.find_element_by_css_selector(".radioBtns [for=\"data\"]")
    dataBtn.click()

    # Wait the page to fresh.
    time.sleep(3)

    sys.stderr.write("Write "+currency+ " to csv file...\n")
    with open("currency/"+"%sto%s.csv" % (currency, "TWD"), 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)

        csvwriter.writerow(["Date", "SellingRate", "BuyingRate","currency"])

        hasMorePages = True

        while hasMorePages:
            items = driver.find_elements_by_css_selector("#inteTable tbody tr")

            for item in items:
                tds = item.find_elements_by_css_selector("td")

                if tds[0].get_attribute("class") != "itemTtitle":
                    continue

                csvwriter.writerow([tds[0].text, tds[1].text, tds[2].text,currency])

            nextBtn = driver.find_element_by_css_selector(".pageNumberBlock .down")
            if "active" in nextBtn.get_attribute("class"):
                nextBtn.click()
            else:
                hasMorePages = False

            time.sleep(1)

    time.sleep(4)
    # Close the browser.
    driver.quit()
    time.sleep(1)
    print(currency+" finish")

Write USD to csv file...


USDfinish


Write CNY to csv file...


CNYfinish


Write JPY to csv file...


JPYfinish


Write EUR to csv file...


EURfinish


Write AUD to csv file...


AUDfinish


In [7]:
yourPath = 'currency/'
allFileList = os.listdir(yourPath)
for file in allFileList:
      if os.path.isdir(os.path.join(yourPath,file)):
        print("I'm a directory: " + file)
allFileList

['AUDtoTWD.csv',
 'CNYtoTWD.csv',
 'EURtoTWD.csv',
 'JPYtoTWD.csv',
 'USDtoTWD.csv']

In [8]:
currency=pd.DataFrame()
for file in allFileList:
    currency=pd.concat((currency,pd.read_csv(yourPath+file)),axis=0)

currency

,Date,SellingRate,BuyingRate,currency
0,2019-01-31,22.27,22.47,AUD
1,2019-02-01,22.22,22.42,AUD
2,2019-02-11,21.69,21.89,AUD
3,2019-02-12,21.76,21.96,AUD
4,2019-02-13,21.83,22.03,AUD
...,...,...,...,...
238,2020-01-17,29.88,29.98,USD
239,2020-01-20,29.90,30.00,USD
240,2020-01-21,29.96,30.06,USD
241,2020-01-22,29.96,30.06,USD


In [9]:
currency.to_csv('currency.csv')